# Regresión lineal simple

### Importar paquetes necesarios

Para este laboratorio necesitarás los siguientes paquetes:
- NumPy  
- Matplotlib  
- Pandas  
- Scikit-learn

In [ ]:
!pip install numpy==2.2.0
!pip install pandas==2.2.3
!pip install scikit-learn==1.6.0
!pip install matplotlib==3.9.3

Ahora puedes importar estas bibliotecas.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# %matplotlib inline

## Cargar los datos

El conjunto de datos que usarás se encuentra en la siguiente URL.

In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%202/data/FuelConsumptionCo2.csv"
df = pd.read_csv(url)

# Ver si se cargaron los datos
df.sample(5)

## Entender los datos

### `FuelConsumption.csv`:
Usarás un conjunto de datos de consumo de combustible, **`FuelConsumption.csv`**, que contiene calificaciones de consumo de combustible específicas por modelo y estimaciones de emisiones de dióxido de carbono para vehículos nuevos de servicio ligero a la venta al público en Canadá. *Fuente del conjunto de datos.*

- **MODEL YEAR** e.g. 2014  
- **MAKE** e.g. VOLVO  
- **MODEL** e.g. S60 AWD  
- **VEHICLE CLASS** e.g. COMPACT  
- **ENGINE SIZE** e.g. 3.0  
- **CYLINDERS** e.g 6  
- **TRANSMISSION** e.g. AS6  
- **FUEL TYPE** e.g. Z  
- **FUEL CONSUMPTION in CITY(L/100 km)** e.g. 13.2  
- **FUEL CONSUMPTION in HWY (L/100 km)** e.g. 9.5  
- **FUEL CONSUMPTION COMBINED (L/100 km)** e.g. 11.5  
- **FUEL CONSUMPTION COMBINED MPG (MPG)** e.g. 25  
- **emisiones de CO₂ (g/km)** e.g. 182  

Tu tarea será crear un **modelo de regresión lineal simple** a partir de **una** de estas **características** para **predecir las emisiones de CO₂** de autos no observados con base en esa característica.

### Explorar los datos

Primero, considera un **resumen estadístico** de los datos.

In [ ]:
df.describe()

A partir de los datos, podemos ver que la mayoría de los autos (aprox. 75%) tienen una eficiencia de combustible entre 11 y 31 MPG. Sin embargo, un auto muestra un valor de 60 MPG, mucho mayor que el resto. Esto podría ser una lectura válida para un vehículo altamente eficiente o híbrido, o podría ser un valor atípico (outlier) o un error de captura de datos.

**MODELYEAR** tiene desviación estándar 0, por lo tanto no contiene información relevante.

#### Seleccionar características

Selecciona algunas características que puedan indicar las emisiones de CO₂ para explorar más.

In [ ]:
cdf = df[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB','CO2EMISSIONS']]
cdf.sample(9)

#### Visualizar características
Considera los histogramas para cada una de estas características.

In [ ]:
viz = cdf[['CYLINDERS','ENGINESIZE','FUELCONSUMPTION_COMB','CO2EMISSIONS']]
viz.hist()
plt.show()

Como puedes ver, la mayoría de los motores tienen 4, 6 u 8 cilindros, y tamaños de motor entre 2 y 4 litros.  
Como cabría esperar, el **consumo de combustible combinado** y las **emisiones de CO₂** tienen distribuciones muy similares.  
Adelante, muestra algunas **gráficas de dispersión** de estas características contra las **emisiones de CO₂**, para ver qué tan lineales son sus relaciones.

In [ ]:
plt.scatter(cdf.FUELCONSUMPTION_COMB, cdf.CO2EMISSIONS, color='blue')
plt.xlabel("FUELCONSUMPTION_COMB")
plt.ylabel("CO2EMISSIONS")
plt.show()

Este es un resultado informativo. Tres grupos de autos presentan una relación lineal fuerte entre su **consumo combinado** y sus **emisiones de CO₂**.  
Sus **interceptos** son similares, mientras que sus **pendientes** difieren notablemente.

In [ ]:
plt.scatter(cdf.ENGINESIZE, cdf.CO2EMISSIONS, color='blue')
plt.xlabel("ENGINESIZE")
plt.ylabel("CO2EMISSIONS")
plt.xlim(0, 27)
plt.show()

#### Ejercicio de práctica 1

Grafica **CYLINDERS** contra **CO2EMISSIONS**, para ver qué tan lineal es su relación.

In [ ]:
# Escribe tu código aquí
# plt.scatter(cdf.CYLINDERS, cdf.CO2EMISSIONS, color='blue')
# plt.xlabel("CYLINDERS")
# plt.ylabel("CO2EMISSIONS")
# plt.show()

### Extraer la característica de entrada y las etiquetas del conjunto de datos

Aunque quizá no sea la elección ideal de característica de entrada, con fines ilustrativos usarás **ENGINESIZE** para predecir **CO2EMISSIONS** con un modelo de **regresión lineal**.  
Puedes comenzar extrayendo las variables de entrada (**X**) y de salida (**y**) del conjunto de datos.

In [ ]:
X = cdf.ENGINESIZE.to_numpy()
y = cdf.CO2EMISSIONS.to_numpy()

#### Crear conjuntos de entrenamiento y prueba

A continuación, dividirás el conjunto de datos en grupos **mutuamente exclusivos** de entrenamiento y prueba. Entrenarás un modelo de **regresión lineal simple** con el conjunto de entrenamiento y estimarás su capacidad de generalización usando el conjunto de prueba.

Como el resultado verdadero de cada punto del conjunto de prueba es conocido, puedes **evaluar** la precisión *out-of-sample* de tu modelo.

Ahora, divide aleatoriamente tus datos en **entrenamiento** y **prueba**, usando 80% para entrenamiento y 20% para prueba. La fracción exacta depende del tamaño de tus datos (típicamente 20–30% para prueba). Cuanto **más pequeño** sea tu conjunto de datos, **más grande** debe ser el conjunto de entrenamiento: en datos pequeños es fácil encontrar patrones espurios. La desventaja es que tu evaluación de generalización tendrá menos confiabilidad. En datos, **más grande suele ser mejor**.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
type(X_train), np.shape(X_train), np.shape(y_train)

### Construir un modelo de regresión lineal simple

Usarás **scikit-learn** para construir el modelo. (Consulta la documentación de *LinearRegression* si deseas más detalles.)

In [ ]:
from sklearn import linear_model

# crear el objeto modelo
regressor = linear_model.LinearRegression()

# entrenar el modelo (scikit-learn espera entrada 2D: (n_observaciones, n_features))
regressor.fit(X_train.reshape(-1, 1), y_train)

# imprimir los parámetros
print('Coefficients:', regressor.coef_[0])   # en regresión simple hay un solo coeficiente
print('Intercept:', regressor.intercept_)

Aquí, el **coeficiente** y el **intercepto** son los parámetros de la regresión que determinó el modelo.  
Definen la **pendiente** y el **intercepto** de la recta de **mejor ajuste** sobre los datos de entrenamiento.

### Visualizar las salidas del modelo

Puedes visualizar qué tan bien **ajusta** el modelo a los datos de entrenamiento graficando la **recta ajustada** sobre los puntos.

El modelo de regresión es la recta:  
$$\hat{y} = \text{Intercept} + \text{Coefficient} \times x$$

In [ ]:
plt.scatter(X_train, y_train, color='blue')
plt.plot(X_train, regressor.coef_ * X_train + regressor.intercept_, '-r')
plt.xlabel("ENGINESIZE")
plt.ylabel("CO2EMISSIONS")
plt.show()

#### Evaluación del modelo

Puedes comparar los valores reales y los **predichos** para calcular la precisión de un modelo de regresión. Las **métricas de evaluación** son clave porque indican en qué áreas mejorar.

Usaremos **MSE** aquí para evaluar el modelo sobre el conjunto de prueba. (También listamos otras métricas comunes.)

- **Mean Absolute Error (MAE):** media del valor absoluto de los errores; fácil de interpretar, es el “error promedio”.  
- **Mean Squared Error (MSE):** media del cuadrado del error; es la métrica que el modelo minimiza para encontrar la mejor recta, por eso también se le llama **suma de residuos al cuadrado**.  
- **Root Mean Squared Error (RMSE):** la raíz cuadrada del MSE; vuelve a las mismas unidades de la variable objetivo, lo que facilita la interpretación.  
- **R2-Score:** no es un error, sino una métrica popular del desempeño; representa qué tan cerca están los puntos a la **recta de regresión** ajustada. El mejor valor es **1.0** y puede ser **negativo**.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# predicciones de prueba
y_pred = regressor.predict(X_test.reshape(-1, 1))

# evaluación
print("Mean absolute error: %.2f" % mean_absolute_error(y_test, y_pred))
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
print("Root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2-score: %.2f" % r2_score(y_test, y_pred))

## Ejercicios de práctica

#### 1. Grafica el resultado del modelo de regresión **sobre los datos de prueba** (en lugar de los de entrenamiento). Evalúa visualmente si el resultado es bueno.

In [ ]:
plt.scatter(...)    # AGREGA CÓDIGO

<details><summary>Haz clic aquí para ver la solución</summary>

```python
plt.scatter(X_test, y_test, color='blue')
plt.plot(X_test, regressor.coef_ * X_test + regressor.intercept_, '-r')
plt.xlabel("ENGINESIZE")
plt.ylabel("CO2EMISSIONS")
plt.show()
```

</details>

Ahora veamos las métricas de evaluación si entrenas un modelo usando la característica **FUELCONSUMPTION_COMB**.

#### 2. Selecciona la característica de consumo de combustible del DataFrame y divide los datos 80%/20% en entrenamiento y prueba.  
Usa el mismo `random_state` que antes para poder comparar objetivamente.

In [ ]:
X = # AGREGA CÓDIGO

X_train, X_test, y_train, y_test = # AGREGA CÓDIGO

<details><summary>Haz clic aquí para ver la solución</summary>

```python
X = cdf.FUELCONSUMPTION_COMB.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

</details>

#### 3. Entrena un modelo de regresión lineal usando los datos de entrenamiento que creaste.  
Recuerda transformar tu característica 1D a un arreglo 2D.

In [ ]:
regr = linear_model.# AGREGA CÓDIGO

# AGREGA CÓDIGO

<details><summary>Haz clic aquí para ver la solución</summary>

```python
regr = linear_model.LinearRegression()
regr.fit(X_train.reshape(-1, 1), y_train)
```

</details>

#### 4. Usa el modelo para hacer **predicciones de prueba** con los datos de consumo de combustible de prueba.

In [ ]:
y_pred = # AGREGA CÓDIGO

<details><summary>Haz clic aquí para ver la solución</summary>

```python
y_pred = regr.predict(X_test.reshape(-1, 1))
```

</details>

#### 5. Calcula e imprime el **MSE** de las predicciones en prueba.

In [ ]:
# AGREGA CÓDIGO

<details><summary>Haz clic aquí para ver la solución</summary>

```python
from sklearn.metrics import mean_squared_error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
```

</details>

Como podrías esperar a partir del análisis exploratorio, el **MSE** es menor cuando entrenamos usando **FUELCONSUMPTION_COMB** en lugar de **ENGINESIZE**.